<a href="https://colab.research.google.com/github/ala-sk98/OpenGait/blob/master/OpenGait_GaitPart_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the Repository:
Clone the opengait repository to your Google Colab workspace. This will give you access to the package and its modules.

In [43]:
!git clone https://github.com/ala-sk98/OpenGait.git

fatal: destination path 'OpenGait' already exists and is not an empty directory.


In [7]:
!pip install einops
!pip install kornia
!pip install tqdm pyyaml tensorboard opencv-python kornia
!pip install torch==1.10 torchvision==0.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 8.7 MB/s eta 0:00:00


In [40]:
import sys
sys.path.append('/content/OpenGait')  # Adjust the path accordingly


In [82]:
from opengait.modeling.models.gaitpart import GaitPart